# Import

In [1]:
import numpy as np
import pandas as pd
import csv
import datetime
import math

# myLibrary

In [2]:
#製造answer.csv的第1列
def answer_header():
    header = ['user_id']
    for i in range(28):
        header.append('time_slot_' + str(i) )
    return header

In [3]:
def get_week(date_str):
    d = get_date(date_str)
    w = datetime.datetime.strftime(d, '%w')
    return int(w)

In [4]:
def get_date(date_str):
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

In [5]:
def get_slot(time_str):
    the_time = datetime.datetime.strptime(time_str, '%H:%M:%S.%f')
    time_sec0 = datetime.datetime.strptime('01:00:00.00', '%H:%M:%S.%f')
    time_sec1 = datetime.datetime.strptime('09:00:00.00', '%H:%M:%S.%f')
    time_sec2 = datetime.datetime.strptime('17:00:00.00', '%H:%M:%S.%f')
    time_sec3 = datetime.datetime.strptime('21:00:00.00', '%H:%M:%S.%f')
    time_sec4 = datetime.datetime.strptime('23:59:59.99', '%H:%M:%S.%f')
    if time_sec0 <= the_time and the_time <= time_sec1:
        slot = 0
    elif time_sec1 <= the_time and the_time <= time_sec2:
        slot = 1
    elif time_sec2 <= the_time and the_time <= time_sec3:
        slot = 2
    elif time_sec3 <= the_time and the_time <= time_sec4:
        slot = 3
    else:
        slot = 4
    return slot

In [6]:
def get_days(end, start):
    d = end - start
    return d.days

In [7]:
def update_week(week, date_str, event_week, addition):
    date = get_date(date_str)
    end_date = get_date('2017-08-14')
    start_date = get_date('2017-05-13')
    if end_date > date and date > start_date:
        days = get_days(end_date, date)
        weight = 0
        for i in range(1,33):
            if (days/7) <= i:
                weight = math.pow(0.5, i)
                break
        addition = list(np.array(addition) * weight)
        week.days[event_week].add_slots(addition)
        another_addition = [0, 0, 0]
        another_addition.append(addition[4])
        week.days[event_week-1].add_slots(another_addition)
    return week

In [8]:
def error_list():
    errorlist= ['error', 'api error', 'limit playzone countdown']
    return errorlist

# myClass

In [9]:
class Day():
    def __init__(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def reset(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def get_slots(self):
        return [self.slot0, self.slot1, self.slot2, self.slot3]
    def add_slots(self, addition):
        self.slot0 += addition[0]
        self.slot1 += addition[1]
        self.slot2 += addition[2]
        self.slot3 += addition[3]

In [10]:
class Week():
    def __init__(self):
        self.days = []
        for i in range(7):
            self.days.append(Day())
    def reset(self):
        for i in range(7):
            self.days[i].reset()
    def get_answer(self, id):
        tmp = []
        for i in range(1, 7):
            tmp += self.days[i].get_slots()
        tmp += self.days[0].get_slots()
        answer = [id] + list(np.array(tmp))
        return answer

# PreProcess Training Data

In [11]:
with open('reference8.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(answer_header())
    week = Week()
    # 開始處理資料
    print(datetime.datetime.now())
    for data_count in range(1, 46): # revise
        # 進度顯示
        print(data_count)
        
        data = pd.read_csv('public/data-0' + str(data_count) + '.csv')
        
        # 取得目前csv檔案的首尾user_id
        head = data.head(1)['user_id'].values[0]
        tail = data.tail(1)['user_id'].values[0]
        
        # 依序處理各id資料並寫入reference.csv
        for id in range(head, tail+1):
            id_data = data[data['user_id'] == id]
            length = len(id_data)
            
            # 依序處理某id的每一筆資料
            now_date = id_data['event_time'].values[0].split()[0]
            addition = [0, 0, 0, 0, 0]
            for i in range(length):
                # 切分時間獲得星期和slot
                event_time = id_data['event_time'].values[i].split()
                date = event_time[0]
                
                # 當日期變化
                if date != now_date:
                    # 將前日的紀錄更新進week
                    event_week = get_week(now_date)
                    week = update_week(week, now_date, event_week, addition)
                    #並初始化now_date和addition
                    now_date = date
                    addition = [0, 0, 0, 0, 0]
                
                # 若無error則把紀錄活動至addition
                #trigger = id_data['action_trigger'].values[i]
                #if trigger not in error_list():
                event_slot = get_slot(event_time[1])
                addition[event_slot] = 1
            
            # 補齊最後一段日期的紀錄
            event_week = get_week(now_date)
            week = update_week(week, now_date, event_week, addition)
            
            # 寫入answer.csv
            writer.writerow(week.get_answer(id))
            
            # 初始化week
            week.reset()
            
            #break # revise

    print(datetime.datetime.now())
# test

2018-12-10 17:48:13.852448
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
2018-12-10 18:44:17.789996


# Main

In [12]:
"wait"

'wait'

# Test Area

In [13]:
type(id_data['played_duration'].values[0])

numpy.int64

In [14]:
data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
0,55880,112463,5488772,16,2017-01-03 13:13:20.344,149,seek,iOS,16,16,cellular,False
1,55880,112463,5488772,16,2017-01-03 13:13:38.593,13,seek,iOS,16,16,cellular,False
2,55880,112463,5488772,16,2017-01-03 13:14:18.986,37,pause,iOS,16,16,cellular,False
3,55880,112463,5488773,16,2017-01-03 23:39:12.573,106,seek,iOS,16,16,cellular,False
4,55880,112463,5488773,16,2017-01-03 23:41:33.405,141,seek,iOS,16,16,cellular,False
5,55880,112463,5488773,16,2017-01-03 23:47:08.388,333,pause,iOS,16,16,cellular,False
6,55880,112463,5488774,16,2017-01-04 01:17:23.340,869,video ended,iOS,16,16,cellular,False
7,55880,112463,5488775,16,2017-01-10 11:43:26.988,2011,pause,iOS,17,18,cellular,False
8,55880,112463,5488776,16,2017-01-10 23:18:11.956,1560,video ended,iOS,17,18,cellular,False
9,55880,112463,5488776,16,2017-01-10 23:28:43.767,617,pause,iOS,18,18,cellular,False


# Check Answer

In [15]:
answer = pd.read_csv('reference8.csv')
answer

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.007812,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000977,0.000000,0.000000,0.000977,0.000000,0.000000,0.000000
1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.125000,0.500000,0.000000
3,3,0.000000,0.000488,0.000000,0.000000,0.000488,0.500488,0.000000,0.000000,0.500000,...,0.500000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.500244,0.000244,0.000000
4,4,0.468750,0.968750,0.000000,0.812500,0.968750,0.968750,0.000000,1.562500,0.968750,...,0.000000,0.328125,0.484375,0.484375,0.328125,0.062500,0.234375,0.484375,0.000000,0.546875
5,5,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000
6,6,0.000488,0.691284,0.000488,0.000610,0.063110,0.507690,0.000244,0.000000,0.000000,...,0.001099,0.125244,0.948608,0.572144,0.500000,1.071533,0.751831,0.813904,0.000000,0.000000
7,7,0.625000,0.562500,0.562500,0.000000,0.687500,0.625000,0.500000,0.000000,0.687500,...,0.625000,0.000000,0.656250,0.812500,0.812500,0.500000,0.500000,0.937500,0.812500,0.000000
8,8,0.000977,0.001953,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000
